In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import google_api_key

# Configure gmaps
gmaps.configure(api_key=google_api_key)

In [34]:
city_file = "city_info.csv"

In [35]:
#Read in CSV file
city_df = pd.read_csv(city_file)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lebu,-37.6167,-73.6500,13.57,74,0,4.89,CL,1619444329
1,Ovsyanka,55.9575,92.5519,9.00,46,0,2.00,RU,1619444330
2,Blagoyevo,63.3667,47.9167,6.95,71,100,2.01,RU,1619444330
3,Alta Floresta,-9.8756,-56.0861,24.00,100,90,1.54,BR,1619444330
4,Témiscaming,46.7212,-79.0971,-1.00,59,0,4.12,CA,1619444331


In [39]:
#Finding max humdity for our max_intensity
city_df.sort_values(by = "Humidity", ascending = False).head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
381,Cooma,-36.2333,149.1333,3.0,100,0,3.09,AU,1619444452
405,Sucúa,-2.4667,-78.1667,19.0,100,90,1.03,EC,1619444460
59,Cap-aux-Meules,47.3821,-61.8597,5.0,100,100,10.80,CA,1619444347
414,Orchard Homes,46.8633,-114.0484,1.0,100,90,1.54,US,1619444463
639,Lake Charles,30.2131,-93.2044,20.0,100,75,1.54,US,1619444535


In [40]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

#Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_map = gmaps.heatmap_layer(locations, weights=city_df["Humidity"], 
                                 dissipating=True, max_intensity=100,
                                 point_radius=10)


# Add layer
fig.add_layer(heat_map)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
#Converting Max Temp column to float so we can use it
city_df["Max Temp"].dtypes
city_df.astype({"Max Temp": "float"})

#Finding ideal weather condition 
weather_condition_df = city_df.loc[(city_df["Max Temp"] > 24) & (city_df["Max Temp"] < 29)]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Wind Speed"] < 10]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Cloudiness"] == 0]
weather_condition_df = weather_condition_df.dropna()
weather_condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,Saint-Philippe,-21.3585,55.7679,27.00,69,0,8.23,RE,1619444346
117,East London,-33.0153,27.9116,25.00,61,0,3.60,ZA,1619444365
161,Awjilah,29.1081,21.2869,27.66,19,0,3.44,LY,1619444379
226,Faanui,-16.4833,-151.7500,26.70,70,0,1.16,PF,1619444399
230,Saint-Pierre,-21.3393,55.4781,27.00,69,0,8.23,RE,1619444401
251,Dwārka,22.2394,68.9678,28.26,74,0,7.20,IN,1619444410
284,Verāval,20.9000,70.3667,28.50,71,0,5.46,IN,1619444420
332,Jalu,29.0331,21.5482,27.68,19,0,3.63,LY,1619444436


In [42]:
#Creating blank column for Hotel Name
weather_condition_df["Hotel Name"] = ""
weather_condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Saint-Philippe,-21.3585,55.7679,27.00,69,0,8.23,RE,1619444346,
117,East London,-33.0153,27.9116,25.00,61,0,3.60,ZA,1619444365,
161,Awjilah,29.1081,21.2869,27.66,19,0,3.44,LY,1619444379,
226,Faanui,-16.4833,-151.7500,26.70,70,0,1.16,PF,1619444399,
230,Saint-Pierre,-21.3393,55.4781,27.00,69,0,8.23,RE,1619444401,
251,Dwārka,22.2394,68.9678,28.26,74,0,7.20,IN,1619444410,
284,Verāval,20.9000,70.3667,28.50,71,0,5.46,IN,1619444420,
332,Jalu,29.0331,21.5482,27.68,19,0,3.63,LY,1619444436,


In [43]:
#Finding the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


#Looping through pandas dataframe to find closest hotel
for index, row in weather_condition_df.iterrows():

    #Coordinates for finding closest hotel
    #marker_locations = weather_condition_df[['Lat', 'Lng']]
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Params for our API request
    params = {
    "location": f"{lat},{lng}",  
    "rankby": "distance",
    "type": "lodging",
    "key": google_api_key,
    }

    # Create url and make API request
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        #Placing hotel name in our dataframe from our API
        weather_condition_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        #print("Missing field/result... skipping.")
        pass


In [44]:
weather_condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Saint-Philippe,-21.3585,55.7679,27.00,69,0,8.23,RE,1619444346,"Chambres d'hôte ""La Trinité"""
117,East London,-33.0153,27.9116,25.00,61,0,3.60,ZA,1619444365,Lynton Court
161,Awjilah,29.1081,21.2869,27.66,19,0,3.44,LY,1619444379,استراحة التعاون أوجلة
226,Faanui,-16.4833,-151.7500,26.70,70,0,1.16,PF,1619444399,Teina Pascale
230,Saint-Pierre,-21.3393,55.4781,27.00,69,0,8.23,RE,1619444401,FLORIRI
251,Dwārka,22.2394,68.9678,28.26,74,0,7.20,IN,1619444410,Hotel Bansi
284,Verāval,20.9000,70.3667,28.50,71,0,5.46,IN,1619444420,Jayesh fofandi
332,Jalu,29.0331,21.5482,27.68,19,0,3.63,LY,1619444436,صاحبة الفخامة


In [45]:
#Convert Hotel Name to list for info box
hotel_names = weather_condition_df["Hotel Name"].tolist()
city_names = weather_condition_df["City"].tolist()
country_names = weather_condition_df["Country"].tolist()

#Contents of our info box
info_box_content = [
    f"Name: {hotel} City: {city} Country: {country}"
    for hotel, city, country 
    in zip(hotel_names, city_names, country_names)
]

In [46]:
# Create a map using state centroid coordinates to set markers
marker_locations = weather_condition_df[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
#fig_2 = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content = info_box_content)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))